In [1]:
import os
import re
import gc
import time
import nltk
import json
import requests
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
pd.set_option("display.max_colwidth", None) 
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from langchain.document_loaders import CSVLoader, DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from langchain_core.documents import Document
import nltk

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
doc = pd.read_csv("C:/Users/lords/OneDrive/Desktop/my_dataset.csv")

In [3]:
def minimal_normalize(text: str) -> str:
    if not isinstance(text, str):
        return text
    text = text.strip()  # удаляем пробелы в начале и конце
    text = re.sub(r'\s+', ' ', text)  # заменяем последовательные пробелы одним
    return text


# Создаем вспомогательные столбцы с нормализованными значениями
doc["question_norm"] = doc["question"].apply(minimal_normalize)
doc["answer_norm"] = doc["answer"].apply(minimal_normalize)

# Удаляем дубликаты по нормализованным колонкам
doc = doc.drop_duplicates(subset=["question_norm", "answer_norm"])

# Если вспомогательные столбцы не нужны, можно их удалить:
doc.drop(columns=["question_norm", "answer_norm"], inplace=True)

print("Количество строк после очистки:", len(doc))

Количество строк после очистки: 16


In [4]:
def fix_case(text):
    if not isinstance(text, str):
        return text
    replacements = {
        r',\s*А\.': ', человек 1',
        r',\s*Б\.': ', человек 2',
        r'\bА\.\s?': 'человек 1 ',
        r'\bБ\.\s?': 'человек 2 '
    }
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text.strip()

# Применяем ко всему DataFrame
doc = doc.apply(lambda col: col.map(fix_case))
doc

,id,question,answer
0,0,"человек 1 придумала идею для сериала и во всех подробностях рассказала ее человек 2 , который впоследствии использовал идею для корткометражного фильма, не упомянув человек 1 в качестве соавтора. Являются ли действия человек 2 правомерными?","Согласно статье 1259 ГК РФ, идея не является объектом авторского права. Идея должна быть воплощена в объективную форму - произведение, которое уже подлежит охране. В запросе о фиксации человек 1 сценария сериала или другой его части не говорится, поэтому сложно дать однозначный ответ. Answer содержит однозначное решение, при этом присутствует комментарий ""соавторства идеи"", что противоречит законодательству(см. п. 5 статьи 1259 ГК РФ)."
3,3,"человек 1 разработала язык программирования ""Матрикс"", который использовался в компании М. для обеспечения высокопроизводительных вычислений на серверах компании. Все сотрудники были в надлежащей форме уведомлены, что данный язык программирования относится к секрету производства, который является интеллектуальной собственностью компании. Впоследствии человек 2 , который работал программистом в этой компании, уволился из компании М. и основал свою собственную фирму, где стал использовать язык программирования ""Матрикс"" в этих же целях. Являются ли действия человек 2 правомерными?","Язык программирования в данном случае является секретом производства (ст. 1465 ГК РФ), созданным человек 1 в ходе выполнения своих трудовых обязанностей и исключительное право на который принадлежит работодателю (компании М., это подчеркнуто в запросе) (служебный секрет производства, ст. 1470 ГК РФ). человек 2 был уведомлен о данном ноу-хау и после увольнения использовал сведения, что является правонарушением (ст. 1472 ГК РФ). Для использования языка программирования человек 2 необходимо получить лицензию (ст. 1469 ГК РФ). Answer содержит правильный ответ, однако не раскрывает особенности прав и обязанностей, связанных с секретами производства."
6,6,"человек 1 написал стихотворение, в котором не используются знаки препинания: ""Сияет звезда в темной ночи Плывут облака легко и стремительно По реке тихо шепчет вода Луна молча смотрит в окно"". человек 2 опубликовал стихотворение в своем блоге под своим именем, но при этом расставил знаки препинания: ""Сияет звезда в темной ночи? Плывут облака легко и стремительно? По реке тихо шепчет вода! Луна молча смотрит в окно..."". Когда человек 1 обвинил человек 2 в плагиате, человек 2 сослался на то, что такое четверостишие может написать кто угодно, так как используются литературные штампы, а при расстановке знаков препинания получилось оригинальное произведение со своим собственным смыслом и оригинальной последовательностью символов. Являются ли действия человек 2 правомерными?","Согласно пункту 1 статьи 1257 Гражданского кодекса Российской Федерации, авторское право на произведение возникает в момент его создания. Это означает, что человек 1 является обладателем авторского права на свое стихотворение. Согласно пункту 2 статьи 1257 Гражданского кодекса Российской Федерации, обладатель авторского права имеет исключительное право использовать произведение в любой форме и любым способом. Другие лица могут использовать произведение только с согласия обладателя авторского права или в случаях, предусмотренных законом. Таким образом, человек 2 не имеет права использовать произведение человек 1 без его согласия или указывать себя в качестве автора. В данном случае, человек 2 не только использовал произведение человек 1 , но и изменил его, добавив свои знаки препинания. Это является нарушением авторских прав и не может быть признано правомерным."
8,8,"человек 1 и человек 2 разработали программное обеспечение для оптимизации делопроизводства, которую они использовали в своей фирме. Для написания кода они использовали язык программирования ""С++"". Впоследствии человек 1 написал аналогичную программу на языке программирования ""Ruby"" для оптимизации делопроизводства, которую без разрешения человек 2 и без упомина

In [5]:
# Определим loader
loader = DataFrameLoader(doc, page_content_column="answer")

documents = loader.load()

documents[0]

Document(metadata={'id': 0, 'question': 'человек 1 придумала идею для сериала и во всех подробностях рассказала ее человек 2 , который впоследствии использовал идею для корткометражного фильма, не упомянув человек 1 в качестве соавтора. Являются ли действия человек 2 правомерными?'}, page_content='Согласно статье 1259 ГК РФ, идея не является объектом авторского права. Идея должна быть воплощена в объективную форму - произведение, которое уже подлежит охране. В запросе о фиксации человек 1 сценария сериала или другой его части не говорится, поэтому сложно дать однозначный ответ. Answer содержит однозначное решение, при этом присутствует комментарий "соавторства идеи", что противоречит законодательству(см. п. 5 статьи 1259 ГК РФ).')

In [6]:
# Устанавливаем punkt для токенизации предложений, если не установлен
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# Создаём адаптивный сплиттер с твоими переменными
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,  # Целевой размер чанка, как ты просил
    chunk_overlap=100,  # Перекрытие для контекста
    length_function=len,
)

# Функция для разбиения с учётом предложений
def split_with_sentences(documents):
    split_documents = []
    for doc in documents:
        # Токенизируем текст на предложения
        sentences = nltk.sent_tokenize(doc.page_content)
        current_chunk = ""
        for sentence in sentences:
            # Если добавление предложения не превышает chunk_size
            if len(current_chunk) + len(sentence) <= 600:
                current_chunk += sentence + " "
            else:
                # Добавляем текущий чанк и начинаем новый
                if current_chunk:
                    split_documents.append(Document(page_content=current_chunk.strip(), metadata=doc.metadata))
                current_chunk = sentence + " "
        # Добавляем остаток
        if current_chunk:
            split_documents.append(Document(page_content=current_chunk.strip(), metadata=doc.metadata))
    return split_documents

# Применяем адаптивное разбиение к твоим данным
split_documents = split_with_sentences(documents)

# Проверяем результат
print("Количество чанков с адаптивным разбиением:", len(split_documents))
print("Пример чанка:", split_documents[0].page_content)

Количество чанков с адаптивным разбиением: 22
Пример чанка: Согласно статье 1259 ГК РФ, идея не является объектом авторского права. Идея должна быть воплощена в объективную форму - произведение, которое уже подлежит охране. В запросе о фиксации человек 1 сценария сериала или другой его части не говорится, поэтому сложно дать однозначный ответ. Answer содержит однозначное решение, при этом присутствует комментарий "соавторства идеи", что противоречит законодательству(см. п. 5 статьи 1259 ГК РФ).


In [7]:
split_documents[15].page_content

'Компьютерная игра, как объект авторского права может интепретироваться как сложный мульдимедийный продукт (1 ст. 1240 ГК РФ), охватывающий различные аспекты создания продукта (сюжет, код ЭВМ, персонажи и т.д). Претензия Б законна согласно п.9 и п.10 ч.2 ст. 1270 ГК РФ ( о незаконной переработанности). Но гражданину Б придется доказать, что переработанная игра — не похожее самостоятельное произведение, а плагиат писателя человек 1 P.S научная статья ('

In [8]:
# Приступим к векторизации документа 
embeddings = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device":"cuda"}
)


In [9]:
# VectorStore: Мы испоьзуем FAISS
db = FAISS.from_documents(
    split_documents, embeddings
)

db.save_local("faiss_db")

In [10]:
# База готова теперь к ней можно делать запросы
db.similarity_search_with_score("я резидент Российской Федерации и хочу подать заявку на изобретение в другой стране ")

[(Document(id='81b76b63-b190-4a86-98a4-a62748df9b4e', metadata={'id': 38, 'question': 'Изобретатель человек 1 является резидентом Российской Федерации, но планирует подать первичную заявку на изобретение в Сингапуре с помощью своего представителя человек 2 , который является резидентом Сингапура. Правомерны ли действия изобретателя человек 1 ?'}, page_content='Порядок патентования за рубежом изобретений и полезных моделей, созданных в Российской Федерации, регламентируется статьей 1395 Гражданского кодекса Российской Федерации, согласно которой зарубежное патентование может быть осуществлено не ранее чем через 6 месяцев после подачи заявки в патентное ведомство России. Данное требование обусловлено необходимостью проверки содержания патентных заявок на наличие в них сведений, составляющих государственную тайну.'),
  0.8490641),
 (Document(id='73bc41e7-fd40-49f0-90aa-5ebf4f2f5a5e', metadata={'id': 38, 'question': 'Изобретатель человек 1 является резидентом Российской Федерации, но плани

In [11]:
# Создаем retriever 
retriever = db.as_retriever(
    search_type = "similarity",
    k=3,
    score_treshold=None,
    
)

In [12]:
# Инициализируем модель Llama 3.2
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    cache_dir="/data/ber/base_models",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    cache_dir="/data/ber/base_models"
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    truncation=True,
)

local_llm = HuggingFacePipeline(pipeline=hf_pipeline)

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [34]:
# Создаем простой шаблон
template = """На основе следующего контекста дай точный и краткий ответ на вопрос, ссылаясь на правовые нормы, если они указаны:

{context}

Вопрос: {question}"""

# Создаем промпт из шаблона 
prompt = ChatPromptTemplate.from_template(template)

# Функция которая будет собирать документ из всех строк 
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Создаем цепочку 
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)

# Определение safe_invoke (добавляем после создания цепочки)
def safe_invoke(query):
    try:
        return chain.invoke(query)
    except Exception as e:
        return f"Ошибка: {str(e)}"


# Использование safe_invoke вместо chain.invoke
result = safe_invoke("Придумал идею для сериала, рассказал ее другу и он украл ее")
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Human: На основе следующего контекста дай точный и краткий ответ на вопрос, ссылаясь на правовые нормы, если они указаны:

Новый собственник получает права целиком и имеет возможность использовать их самостоятельно или передавать третьей стороне во временное пользование по своему усмотрению.

Язык программирования в данном случае является секретом производства (ст. 1465 ГК РФ), созданным человек 1 в ходе выполнения своих трудовых обязанностей и исключительное право на который принадлежит работодателю (компании М., это подчеркнуто в запросе) (служебный секрет производства, ст. 1470 ГК РФ). человек 2 был уведомлен о данном ноу-хау и после увольнения использовал сведения, что является правонарушением (ст. 1472 ГК РФ). Для использования языка программирования человек 2 необходимо получить лицензию (ст. 1469 ГК РФ).

Простое упоминание пункта 2 трудового договора, который подписал человек 1 , не является достаточным доказательством того, что все интеллектуальные права на произведения челове

In [14]:
question = "Являюсь резидентом Российской Федерации, но планирую подать первичную заявку на изобретение за рубежом"
context = db.similarity_search(question)
print(f"""
Answer the question based only on the following context:
{format_docs(context)}

Question: {question}
""")


Answer the question based only on the following context:
Порядок патентования за рубежом изобретений и полезных моделей, созданных в Российской Федерации, регламентируется статьей 1395 Гражданского кодекса Российской Федерации, согласно которой зарубежное патентование может быть осуществлено не ранее чем через 6 месяцев после подачи заявки в патентное ведомство России. Данное требование обусловлено необходимостью проверки содержания патентных заявок на наличие в них сведений, составляющих государственную тайну.

Также заявителю необходимо убедиться в том, что его заявка не нарушает требований соблюдения секретности и экспортных ограничений, предусмотренных Законом РФ от 21.07.93 № 5485-1 «О государственной тайне», согласно которым вся ответственность за разглашение сведений, составляющих государственную тайну, лежит на владельце информации. В рассматриваемом случае таким владельцем является заявитель, подающий заявку за рубежом.

Несмотря на спорность факта о том, что запись с видеоре

In [15]:
# 5. Реализация этапа TRAQ (Trustworthy Retrieval-Augmented Questions)
def traq_check(answer: str) -> str:
    if not answer.strip():
        return "[TRAQ Warning: Ответ пустой]"
    if "ст. " not in answer and "ГК РФ" not in answer:
        return answer + "\n[TRAQ Warning: Возможно, ответ не ссылается на правовые нормы]"
    return answer

In [16]:
rag_chain_extended = (
    {"context": (lambda q: format_docs(retriever.get_relevant_documents(q))), "question": RunnablePassthrough()}
    | prompt
    | (lambda ans: traq_check(str(ans)))         # Этап TRAQ
    | StrOutputParser()                 
)


new_query = "Интелектульное право"
final_answer = rag_chain_extended.invoke(new_query)
print("Сгенерированный ответ с TRAQ:")
print(final_answer)

Сгенерированный ответ с TRAQ:
messages=[HumanMessage(content='На основе следующего контекста дай точный и краткий ответ на вопрос, ссылаясь на правовые нормы, если они указаны:\n\nНесмотря на спорность факта о том, что запись с видеорегистратора является аудиовизуальным произведением (ст. 1263 ГК РФ), так как зачастую они не созданы творческим трудом, согласно судебной практике видеосюжеты относят к объектам авторского права, поэтому человек 1 обладает исключительным правом на видео (ст. 1229 ГК РФ). Для использования видео новостным агенствам необходимо было заключить лицензионный договор с человек 1 (ст. 1235 ГК РФ). человек 1 имеет право на защиту исключительных прав, включающую возмещение убытков (ст. 1252 ГК РФ).\n\nCогласно ст.1228 ГК РФ, автором результата интеллектуальной деятельности признается гражданин, творческим трудом которого создан такой результат. Не признаются авторами результата интеллектуальной деятельности граждане, не внесшие личного творческого вклада в создание 

C:\Users\lords\AppData\Local\Temp\ipykernel_18580\3806609773.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  {"context": (lambda q: format_docs(retriever.get_relevant_documents(q))), "question": RunnablePassthrough()}
